# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts


## Define machine and code 

In [4]:
# facility='cori'
# img_size=64
# run_type='3dgan'
# assert facility in ['cori','summit'], "Error%s"%(facility)
# assert img_size in [64,128,512]
# assert run_type in ['2dgan','2dcgan','3dgan','3dcgan']

In [5]:
def f(facility,img_size,run_type): return facility,img_size,run_type
w=interactive(f,
                facility=ToggleButtons(options=['cori','summit']),
                img_size=ToggleButtons(options=[64,128,512]),
                run_type=ToggleButtons(options=['2dgan','2dcgan','3dgan','3dcgan']) )
display(w)

interactive(children=(ToggleButtons(description='facility', options=('cori', 'summit'), value='cori'), ToggleB…

In [28]:
result=w.result
facility,img_size,run_type=result
print(facility,img_size,run_type)

cori 128 2dcgan


## Read ref_launch.yaml and define dictionary

In [29]:
# start_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts'
launch_cfile=start_dir+'/ref_launch.yaml'

with open(launch_cfile) as f:
    config_dict= yaml.load(f, Loader=yaml.SafeLoader)

In [30]:
## Read ref_launch.yaml
main_code_dir=config_dict[facility]['main_code_dir']
dict_pars=config_dict[facility][img_size][run_type]
dict_pars.update({'nodes':1,'job_name':'analysis_'+run_type,
                 'queue':'debug','bins':'unenven','cores':64,'time':'00:30:00'})

dict_pars['code_dir']=main_code_dir+dict_pars['code_dir']
dict_pars['staging_loc']=config_dict[facility]['staging_loc']
print(dict_pars)
staging_loc=dict_pars['staging_loc']
op_fldr=dict_pars['staging_loc']+dict_pars['op_dir']
op_fldr

{'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/DDP', 'op_dir': '128sq/', 'config': 'config_2d_cgan.yaml', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/', 'nodes': 1, 'job_name': 'analysis_2dcgan', 'queue': 'debug', 'bins': 'unenven', 'cores': 64, 'time': '00:30:00', 'staging_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'}


In [32]:
%%bash -s "$op_fldr"
ls $1

20210205_062936_cgan_predict_0.65_m2
20210205_063709_cgan_predict_1.1_m2
20210205_120608_cgan_predict_0.5_m2
20210205_122504_cgan_predict_0.8_m2
20210209_075831_lambda2.0
20210408_064020_old_gan_lambda2.0
20210408_161912_train_newloss_noddpcode_works
20210411_063412_ddp_code_notusingddp
20210414_102623_single_gpu_fmloss_80
20210414_154956_1node_8_fmloss
20210414_182903_cgan_ddp_1node_2
20210414_82846_2nodes_16_fmloss
20210415_140527_gclip1.0
20210415_151529_noclip
20210415_170935_optD_infront
20210415_180935_cgan_optD_infront
20210415_205808_cgan_bs64_60epochs
20210416_215540_2dgan_fm1.0
20210416_220730_2dcgan_nofm
20210419_131244_2dgan
20210419_131246_2d_cgan
simple_gan_best_runs


In [33]:
ip_folder='20210419_131246_2d_cgan'

dict_pars['ip_folder']=dict_pars['staging_loc']+dict_pars['op_dir']+ip_folder

In [34]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir={code_dir}

python $code_dir/../post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [35]:
fname='batch_analysis_{0}.sh'.format(run_type)
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis_2dcgan.sh


In [36]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [37]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_2dcgan

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/DDP

python $code_dir/../post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210419_131246_2d_cgan  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/ -c 64 -bin unenven
conda deactivate
echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [38]:
%%bash -s "$filename" ## Use python variable in bash
# module load cgpu
sbatch $1

Submitted batch job 41752547


In [27]:
pwd

'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results'